In [ ]:
!python --version

In [65]:
import pandas as pd

In [66]:
import yaml

In [67]:
import time

In [68]:
from datetime import datetime

In [69]:
print(datetime.now())

2022-03-08 17:30:09.265336


In [70]:
f = open("Milestone2B1_Log.txt", "w")

In [71]:
#opening in yaml
data =""
with open('Milestone2\MileStone2B.yaml') as file:
    content = yaml.safe_load_all(file)
    data=next(content)
print(datetime.now())
f.write(f"{datetime.now()};M2B_Workflow Entry \n")
main_flow = data['M2B_Workflow']

2022-03-08 17:30:09.524426


In [72]:
#execting time function
def TimeFunction(inputs,path,task,condition):
    lock.acquire()
    print(condition)
    #cond = ""
    if(len(condition)>0):
        i = 1
        if(condition[len(condition)-2] == '1'):
            i=2
        s=condition[2:len(condition)-4-i]
        print(s)
        print(returnDict[s])
        if(condition[len(condition)-2] == '1'):
            num = eval(condition[len(condition)-2]+condition[len(condition)-1])
        else:
            num = eval(condition[len(condition)-1])
        if(condition[len(condition)-i-2] == '>'):
            
            if returnDict[s]>num:
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return
        elif condition[len(condition)-2-i] == '<':
            if(returnDict[s]<num):
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return
        #cond = eval(s1)
        #print(cond)
        #cond = eval(s1)
        #if cond:
         #   print("true")
        #else:
         #   print("false")
    f.write(f"{datetime.now()};{path} Executing TimeFunction ({inputs['FunctionInput']},{inputs['ExecutionTime']}) \n")
    #print(execution_time)
    print("executing ......")
    time.sleep(int(inputs['ExecutionTime']))
    print("done...")
    if(len(task) == 0):
        f.write(f"{datetime.now()};{path} Exit \n")
    else:
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    lock.release()

In [73]:
def DataLoad(inputs,path,task,condition):
    lock.acquire()
    print(condition)
    #cond=""
    if(len(condition)>0):
        i = 1
        if(condition[len(condition)-2] == '1'):
            i=2
        s=condition[2:len(condition)-4-i]
        print(s)
        print(returnDict[s])
        if(condition[len(condition)-2] == '1'):
            num = eval(condition[len(condition)-2]+condition[len(condition)-1] )
        else:
            num = eval(condition[len(condition)-1])
        #s=condition[2:len(condition)-5]
        #cond = eval(s1)
        #print(s)
        #print(returnDict[s])
        #num = eval(condition[len(condition)-1])
        if(condition[len(condition)-2-i] == '>'):
            
            if returnDict[s]>num:
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return 
        elif condition[len(condition)-2-i] == '<':
            if(returnDict[s]<num):
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return 
        #cond = eval(s1)
        #print(cond)
        #if cond:
          #  print("true")
        #else:
           # print("false")
    print(inputs['Filename'])
    f.write(f"{datetime.now()};{path} Executing DataLoad ({inputs['Filename']}) \n")
    df = pd.read_csv(f"Milestone2\{inputs['Filename']}")
    if(len(task) == 0):
        f.write(f"{datetime.now()};{path} Exit \n")
    else:
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    print("done.....")
    lock.release()
    
    return df,len(df)
    

In [74]:
import threading

In [75]:
lock = threading.Lock()

In [76]:
returnDict = {}

In [77]:
#for each task
def activity(activities,task,path):
    
    f.write(f"{datetime.now()};{path}.{task} Entry \n")
    print(task+"........")
    if(activities[task]['Type'] == 'Task'):
            #print(activities[task],activities[task].get('Condition'))
            if activities[task].get('Condition') is None:
                condition = ""
            else:
                condition = activities[task]['Condition']
                #print(condition)
        
            fun = eval(activities[task]['Function'])
            if activities[task]['Function'] == "DataLoad" :
                returnDict[path+'.'+task+'.NoOfDefects'] = fun(activities[task]['Inputs'],path+"."+task,"",condition)[1]
            else:
                fun(activities[task]['Inputs'],path+"."+task,"",condition)
            #fun(activities[task]['Inputs'],path+"."+task,"")
    elif activities[task]['Type'] == 'Flow':
        if activities[task]['Execution'] == 'Sequential':
                Sequential(activities[task],path+"."+task)
        elif activities[task]['Execution'] == 'Concurrent':
                Concurrent(activities[task],path+"."+task)
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    

In [78]:
def Concurrent(main_flow,path):
    activities = main_flow['Activities']
    
    thread= []
    
    for task in activities:
        #f.write(f"{datetime.now()};{path}.{task} Entry \n")
        t = threading.Thread(target=activity,args=(activities,task,path))
        t.start()
        thread.append(t)
    
    for t in thread:
        t.join()
        

In [79]:
def Sequential(main_flow,path):
    activities = main_flow['Activities']
    #print(main_flow)

    for task in activities:
        #print(task)
        f.write(f"{datetime.now()};{path}.{task} Entry \n")
        print(task," Entry .......")

        if activities[task]['Type'] == 'Task':
            print(activities[task],activities[task].get('Condition'))
            if activities[task].get('Condition') is None:
                condition = ""
            else:
                condition = activities[task]['Condition']
                print(condition)
            fun = eval(activities[task]['Function'])
            if activities[task]['Function'] == "DataLoad" :
                print(path+'.'+task+'.NoOfDefects')
                returnDict[path+'.'+task+'.NoOfDefects'] = fun(activities[task]['Inputs'],path+"."+task,"",condition)[1]
            else:
                fun(activities[task]['Inputs'],path+"."+task,"",condition)
        elif activities[task]['Type'] == 'Flow':
            if activities[task]['Execution'] == 'Sequential':
                Sequential(activities[task],path+"."+task)
            elif activities[task]['Execution'] == 'Concurrent':
                Concurrent(activities[task],path+"."+task)
            f.write(f"{datetime.now()};{path}.{task} Exit \n")
        
        

In [80]:
if main_flow['Execution'] == 'Sequential':
    Sequential(main_flow,"M2B_Workflow")
    f.write(f"{datetime.now()};M2B_Workflow Exit\n")

TaskA  Entry .......
{'Type': 'Task', 'Function': 'DataLoad', 'Inputs': {'Filename': 'Milestone2B_DataInput1.csv'}, 'Outputs': ['DataTable', 'NoOfDefects']} None
M2B_Workflow.TaskA.NoOfDefects

Milestone2B_DataInput1.csv
done.....
FlowA  Entry .......
TaskB........
$(M2B_Workflow.TaskA.NoOfDefects) > 0
M2B_Workflow.TaskA.NoOfDefects
24
true
executing ......
TaskC........
TaskD........
FlowB........
TaskE  Entry .......
{'Type': 'Task', 'Function': 'DataLoad', 'Inputs': {'Filename': 'Milestone2B_DataInput2.csv'}, 'Outputs': ['DataTable', 'NoOfDefects']} None
M2B_Workflow.FlowA.FlowB.TaskE.NoOfDefects
FlowC........
TaskG  Entry .......
{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskG_Input', 'ExecutionTime': '0'}} None
TaskI........
done...

Milestone2B_DataInput1.csv
done.....
$(M2B_Workflow.TaskA.NoOfDefects) < 10
M2B_Workflow.TaskA.NoOfDefects
24
M2B_Workflow.FlowA.TaskD skipped
false

Milestone2B_DataInput2.csv
done.....
TaskF  Entry .......
{'Type': 'T